In [1]:
# preliminary code

# enable hot reload of imported code
%load_ext autoreload
%autoreload 2

In [3]:
# enable src imports if needed (setting an env variable may be enough too)
import sys
sys.path.append("..")

In [ ]:
import pickle
from src.data.datasets.museums3k.utils import Museums3kDataset
from src.data.feature_extractor import *
from src.data.feature_utils import DigitalExhibitionFeatureSet

In [7]:
feature_output_dir_name = 'museums3k'
text_split_strategy ='sentence_by_sentence'
dataset = 'museums3k'
feature_set_name = "base_sentence_open_clip"

In [8]:
textExtractor = TextExtractor.OPEN_CLIP
povExtractor = ImageExtractor.OPEN_CLIP
imageExtractor = ImageExtractor.NONE
videoExtractor = VideoExtractor.NONE

feature_set = DigitalExhibitionFeatureSet(feature_output_dir_name, feature_set_name)
feature_set.set_dataset_name(dataset)

split_strategy = TextSplitStrategy.BY_SENTENCE

museumDataset = Museums3kDataset()
museum_id_list = museumDataset.getMuseumList()

In [ ]:

# description_feature_dict should be a dictionaty id - tensors (#sentences, 512)
# scene_feature_dict should be a dict id - list of rooms, each one is a tensor (#screenshots, 512)
# image_artworks_feature_dict should be a dict id - tensor (1,512)
# video_artworks_feature_dict should be a dict with a features field with aa dict id - tensor (512)

description_feature_dict = {}
scene_feature_dict = {}
image_artworks_feature_dict = {}
video_artworks_feature_dict = {'features':{}}

for id in museum_id_list:
    desc_tensor, scene_img_tensor, index = museumDataset[museumDataset.id_to_idx(id)]
    description_feature_dict[id] = desc_tensor
    scene_feature_dict[id] = list(scene_img_tensor.split(museumDataset.images_per_room))
    s = scene_feature_dict[id][0].shape
    assert all([el.shape == s for el in scene_feature_dict[id]])

base_path_features = os.path.join('..', 'src', 'data', 'features', feature_output_dir_name)


description_feature_dict_fn = "museumDescSentenceLevelSplit_OPEN_CLIP.pickle"
scene_feature_dict_fn = "museumSceneFeatures_OPEN_CLIP.pickle"
image_artworks_feature_dict_fn = "imageArt.pickle"
video_artworks_feature_dict_fn = "videoArt.pickle"

with open(os.path.join(base_path_features, description_feature_dict_fn), 'wb') as f:
    pickle.dump(description_feature_dict, f)
with open(os.path.join(base_path_features, scene_feature_dict_fn), 'wb') as f:
    pickle.dump(scene_feature_dict, f)
with open(os.path.join(base_path_features, image_artworks_feature_dict_fn), 'wb') as f:
    pickle.dump(image_artworks_feature_dict, f)
with open(os.path.join(base_path_features, video_artworks_feature_dict_fn), 'wb') as f:
    pickle.dump(video_artworks_feature_dict, f)


In [10]:
feature_set.add_and_store_descriptions(description_feature_dict_fn, textExtractor, split_strategy)
feature_set.add_and_store_povs(scene_feature_dict_fn, povExtractor)
feature_set.add_and_store_image_art(image_artworks_feature_dict_fn, imageExtractor)
feature_set.add_and_store_video_art(video_artworks_feature_dict_fn, videoExtractor)